In [1]:
%pip install ollama chromadb langchain-community langchain-text-splitters 
%pip install -U langchain-community pypdf
%pip install -U langchain-chroma


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings



In [5]:
# This script demonstrates how to load a PDF document, split it into chunks, 
# --- 1. Load the Document ---
file_path = "C:\\vishnu\\AI ML NLP\\Attention.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
 
# --- 2. Split the Document into Chunks ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
 
print(f"Split document into {len(chunks)} chunks.")
 
# --- 3. Generate Embeddings with Ollama ---
# Initialize the Ollama embedding model
# Make sure your Ollama server is running!
# 'mxbai-embed-large' is a strong open-source choice.
embeddings_model = OllamaEmbeddings(model="mxbai-embed-large")
 
# Get the text from each chunk
#chunk_texts = [chunk.page_content for chunk in chunks]
#embeddings = embeddings_model.embed_documents(chunk_texts)

CHROMA_DB_PATH = "C:\\vishnu\\AI ML NLP\\chroma_db"
COLLECTION_NAME = "pdf_embedding_collection"

# 4. Initialize or load Chroma vectorstore and add documents + embeddings

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model,
    persist_directory=CHROMA_DB_PATH,
    collection_name=COLLECTION_NAME,
)


Split document into 40 chunks.


In [ ]:
# 5. Persist the vectorstore to disk for later retrieval   //no need since persist_directory was provided in vectorestore api
#vectorstore.persist()

print(f"Successfully embedded {len(chunks)} chunks and saved to ChromaDB at {CHROMA_DB_PATH}")

'''"Define a system prompt.py"
# 6. Query the vectorstore
query = "What is the attention?"
docs = vectorstore.similarity_search(query=query, k=2)
print(f"Found {len(docs)} relevant documents for query: {query}")
'''

In [6]:
%pip install -U langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

# ---- CONFIGURATION ----
#CHROMA_DIR = './chroma_db'
#COLLECTION_NAME = 'my_docs'
EMBED_MODEL = 'mxbai-embed-large' #'nomic-embed-text'   # Or your chosen embedding model
LLM_MODEL = 'llama2'             # Your local Ollama LLM model
# ------------------------

# 1. Set up Embedding Model
embeddings = OllamaEmbeddings(model=EMBED_MODEL)

# 2. Connect to Chroma VectorDB
vectorstore = Chroma(
    persist_directory=CHROMA_DB_PATH,
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME
)

# 3. Set up Retriever from vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# 4. Set up LLM (Desktop Ollama)
llm = ChatOllama(model=LLM_MODEL)

# 5. Set up RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  # Optional: to see which docs contributed to the answer
)

# Main Query Loop
print("RAG Agent Ready! (Type 'exit' to quit)\n")
while True:
    user_query = input("Ask a question about your documents: ")
    if user_query.lower() in ["exit", "quit"]:
        break
    result = qa_chain.invoke({"query": user_query})
    print("\n---- Answer ----")
    print(result['result'])
    print("\n---- Sources ----")
    for doc in result['source_documents']:
        print(f"Page: {doc.metadata.get('page', 'N/A')}, Snippet: {doc.page_content[:120].replace('\n', ' ')}")
    print("--------------------\n")


RAG Agent Ready! (Type 'exit' to quit)



OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama2`.